In [38]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import sys
import glob
import pickle
import collections
import pandas

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
import astropy.coordinates as coord
import astropy.units as u
import math
from astropy.table import Table, hstack, vstack
from scipy.io.idl import readsav
import os
import pandas as pd

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.transforms import Bbox,TransformedBbox
from matplotlib.image import BboxImage
from matplotlib.legend_handler import HandlerBase
from matplotlib._png import read_png
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

params = {
    'font.family'        : 'sans',
    'font.size'          : 17,
    'axes.labelsize'     : 20,
    'ytick.labelsize'    : 16,
    'xtick.labelsize'    : 16,
    'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]
parula_0 = ListedColormap(_parula_data, name='parula_0')
parula_0.set_bad((1,1,1))
parula_r = ListedColormap(_parula_data[::-1], name='parula_r')

willi_blau = [0.0722666667, 0.4886666667, 0.8467]


In [2]:
debug = False

## Galpy initialization

In [3]:
import galpy
from galpy.potential import MWPotential2014 as pot
#from galpy.potential import McMillan17 as pot
from galpy.actionAngle import actionAngleStaeckel
from galpy.util import bovy_coords
from galpy.orbit import Orbit

#galpy scale units:                                                                                                                                                                                         
_REFR0 = 8.178 #[kpc] --> galpy length unit, reference: https://arxiv.org/abs/1904.05721                                                                                                                                                   
_REFV0 = 229.0 #[km/s] --> galpy velocity unit, reference: https://arxiv.org/abs/1810.09466

aAS = actionAngleStaeckel(
        pot   = pot,        #potential                                                                                                                                                                      
        delta = 0.45,       #focal length of confocal coordinate system                                                                                                                            
        c     = True        #use C code (for speed)                                                                                                                                                         
        )

print("galpy scale units are _REFR0 = "+str(_REFR0)+" kpc and _REFV0 = "+str(_REFV0)+" km/s.")
#print("The Sun has an angular momentum of "+str(_REFR0 * _REFV0)+" kpc km/s in this action framework with MWPotential2014")
print("The Sun has an angular momentum of "+str(_REFR0 * _REFV0)+" kpc km/s in this action framework with McMillan17")

#Galactocentric position of the Sun according to Gravity Collaboration / Bennett & Bovy (2016)
X_gc_sun_kpc = _REFR0 #[kpc]
Z_gc_sun_kpc = 0.0208 #[kpc]

#(RA = 17:45:37.224 h:m:s, Dec = −28:56:10.23 deg) (Reid& Brunthaler 2004

print(r"We place Sgr A at (x; y; z) = (R_0; 0; z_0) kpc, where")
#print("R_0 = "+str(X_gc_sun_kpc)+" kpc and z_0 = "+str(Z_gc_sun_kpc)+" pc (Bland-Hawthorn & Gerhard, 2016)")
print("R_0 = "+str(X_gc_sun_kpc)+" kpc and z_0 = "+str(Z_gc_sun_kpc)+" pc (Bennett & Bovy 2019)")

#Velocity of the Sun w.r.t. the Local Standard of Rest (e.g. Schoenrich et al. 2010):
U_LSR_kms = 11.10  # [km/s]
V_LSR_kms = 12.24 # [km/s]
W_LSR_kms = 7.25  # [km/s]

#Galactocentric velocity of the Sun:                                                                                                                                                                        
vX_gc_sun_kms =  U_LSR_kms           # = -U              [km/s]
vY_gc_sun_kms =  V_LSR_kms+_REFV0    # = V+v_circ(R_Sun) [km/s]
vZ_gc_sun_kms =  W_LSR_kms           # = W               [km/s]

print("The Sun's velocity with respect to a co-located particle on a circular orbit is")
print("V_LSR = (U_sun, V_sun, W_sun) = ("+str(vX_gc_sun_kms)+", "+str(vY_gc_sun_kms-_REFV0)+", "+str(vZ_gc_sun_kms)+") km/s (Schoenrich 2010)")


galpy scale units are _REFR0 = 8.178 kpc and _REFV0 = 229.0 km/s.
The Sun has an angular momentum of 1872.7620000000002 kpc km/s in this action framework with McMillan17
We place Sgr A at (x; y; z) = (R_0; 0; z_0) kpc, where
R_0 = 8.178 kpc and z_0 = 0.0208 pc (Bennett & Bovy 2019)
The Sun's velocity with respect to a co-located particle on a circular orbit is
V_LSR = (U_sun, V_sun, W_sun) = (11.1, 12.240000000000009, 7.25) km/s (Schoenrich 2010)


### Input of 6D information in observable dimensions

In [4]:
file_directory = '/Users/lspina/Projects/Clusters_surveys/membership_analysis/tables/'


In [5]:
cluster_input = Table.read(file_directory+'clusters_parameters.csv', format='csv')

f = cluster_input['N_RV_mem'] > 1
median_RV_se = np.median(cluster_input['RV_se'][f])
cluster_input['RV_se'][f] = median_RV_se

f = cluster_input['N_RV_mem'] > 0
clusters = cluster_input[f]
nr_clusters = len(clusters)
print("Selected number of clusters")
print(nr_clusters)


/Users/lspina/anaconda2/envs/py36/lib/python3.6/site-packages/astropy/io/ascii/fastbasic.py:120: AstropyWarning: OverflowError converting to FloatType in column Cl_dist_mean, possibly resulting in degraded precision.
  data, comments = self.engine.read(try_int, try_float, try_string)

/Users/lspina/anaconda2/envs/py36/lib/python3.6/site-packages/astropy/io/ascii/fastbasic.py:120: AstropyWarning: OverflowError converting to FloatType in column Cl_dist_median, possibly resulting in degraded precision.
  data, comments = self.engine.read(try_int, try_float, try_string)

/Users/lspina/anaconda2/envs/py36/lib/python3.6/site-packages/astropy/io/ascii/fastbasic.py:120: AstropyWarning: OverflowError converting to FloatType in column Cl_dist_std, possibly resulting in degraded precision.
  data, comments = self.engine.read(try_int, try_float, try_string)

/Users/lspina/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:746: UserWarning: Warning: 'partition' will ignore th

In [6]:
six_dimensions = {}

# Right ascension [deg]
six_dimensions['name'] = clusters['Cluster']
six_dimensions['ra'] = clusters['ra_median']
# Declination [deg]
six_dimensions['dec'] = clusters['dec_median']

# Distance from Sun [pc]
six_dimensions['distance'] = clusters['dist_median']
# Parallax [mas]
six_dimensions['parallax'] = clusters['plx_median']

# Total proper motion in direction of right ascension [mas/yr]
six_dimensions['pmra'] = clusters['pmra_median']
# Total proper motion in direction of declination [mas/yr]
six_dimensions['pmdec'] = clusters['pmdec_median']
# Radial velocity [km/s]
six_dimensions['vrad'] = clusters['RV_median']

In [7]:
e_six_dimensions = {}

# Error of right ascension [mas] to [deg]
e_six_dimensions['ra'] = clusters['ra_std']/(1000.*3600.)
# Error of declination [mas] to [deg]
e_six_dimensions['dec'] = clusters['dec_std']/(1000.*3600.)
# Error of distance from Sun [pc]
e_six_dimensions['distance'] = clusters['dist_std']
    # We are currently sampling a 2-sided Gaussian because Bailer-Jones are only giving 16th/50th/86th percentiles.
    # Any idea how to improve that because of missing posteriors from Bailer-Jones?
# Error of parallax [mas]
e_six_dimensions['parallax']  = clusters['plx_std']
# Error of total proper motion in direction of right ascension [mas/yr]
e_six_dimensions['pmra'] = clusters['pmra_std']
# Error of total proper motion in direction of declination [mas/yr]
e_six_dimensions['pmdec'] = clusters['pmdec_std']
# Error of radial velocity [km/s]
e_six_dimensions['vrad'] = clusters['RV_std']

## Monte Carlo sampling of Orbits

In [8]:
MC_size = 10000

np.random.seed(123)

XYZ_labels       = ['X_XYZ','Y_XYZ','Z_XYZ']
UVW_labels       = ['U_LSR','V_LSR','W_LSR']

Rphiz_labels     = ['R_Rzphi','phi_Rzphi','z_Rzphi']
vRphiz_labels    = ['vR_Rzphi','vphi_Rzphi','vz_Rzphi']

action_labels    = ['J_R','L_Z','J_Z']
ext_orbit_labels = ['ecc', 'zmax', 'R_peri', 'R_ap', 'Energy', 'Rguid']

orbit_labels = np.concatenate((
    XYZ_labels,
    UVW_labels,
    Rphiz_labels,
    vRphiz_labels,
    action_labels,
    ext_orbit_labels    
    ))
print(orbit_labels)

['X_XYZ' 'Y_XYZ' 'Z_XYZ' 'U_LSR' 'V_LSR' 'W_LSR' 'R_Rzphi' 'phi_Rzphi'
 'z_Rzphi' 'vR_Rzphi' 'vphi_Rzphi' 'vz_Rzphi' 'J_R' 'L_Z' 'J_Z' 'ecc'
 'zmax' 'R_peri' 'R_ap' 'Energy' 'Rguid']


### Samples

In [14]:
def sample_6d_uncertainty(
    six_dimensions,
    e_six_dimensions,
    MC_size=MC_size
    ):
    
    """
    This function samples the 6D space with the given uncertainties.
    3 Options are available:
    
    if MC_size==1: assume no uncertainties
    
    if no_correlation==True: Sample 6D parameters independently
    
    if no_correlation==False: Use Gaia DR2 covariance matrix to sample 5D
    and GALAH vrad for 6th D
    """

    np.random.seed(123)
    
    MC_sample_6D = {}
    
    # Option 1: We assume no errors and simply return the actual parameters
    if MC_size == 1:
        for each_key in six_dimensions.keys():
            if each_key == 'distance':
                MC_sample_6D[each_key] = np.array([[six_dimensions[each_key][x]] for x in range(nr_clusters)])
            else:
                MC_sample_6D[each_key] = np.array([[six_dimensions[each_key][x]] for x in range(nr_clusters)])
    
    else:
            # Option2: We sample the errors including the covariance matrix
        MC_sample_6D['ra']       = np.array([np.random.normal(loc=six_dimensions['ra'], scale=e_six_dimensions['ra']) for i in range(MC_size)]).T
        MC_sample_6D['dec']      = np.array([np.random.normal(loc=six_dimensions['dec'], scale=e_six_dimensions['dec']) for i in range(MC_size)]).T
        MC_sample_6D['distance'] = np.array([np.random.normal(loc=six_dimensions['distance'], scale=e_six_dimensions['distance']) for i in range(MC_size)]).T

        MC_sample_6D['pmra']     = np.array([np.random.normal(loc=six_dimensions['pmra'], scale=e_six_dimensions['pmra']) for i in range(MC_size)]).T
        MC_sample_6D['pmdec']    = np.array([np.random.normal(loc=six_dimensions['pmdec'], scale=e_six_dimensions['pmdec']) for i in range(MC_size)]).T
        MC_sample_6D['vrad']     = np.array([np.random.normal(loc=six_dimensions['vrad'], scale=e_six_dimensions['vrad']) for i in range(MC_size)]).T
        
    return MC_sample_6D

## Compute orbit information

In [36]:
# The final orbit information will go into a dictionary, which we initialise with np.nan values

orbit_information = collections.OrderedDict()

for each_orbit_label in orbit_labels:
    orbit_information[each_orbit_label] = np.zeros(nr_clusters); orbit_information[each_orbit_label][:]=np.nan
    orbit_information[each_orbit_label+'_5'] = np.zeros(nr_clusters); orbit_information[each_orbit_label+'_5'][:]=np.nan
    orbit_information[each_orbit_label+'_50'] = np.zeros(nr_clusters); orbit_information[each_orbit_label+'_50'][:]=np.nan
    orbit_information[each_orbit_label+'_95'] = np.zeros(nr_clusters); orbit_information[each_orbit_label+'_95'][:]=np.nan
    orbit_information[each_orbit_label+'_err'] = np.zeros(nr_clusters); orbit_information[each_orbit_label+'_err'][:]=np.nan


In [44]:
def estimate_orbit_parameters(MC_sample_6D, orbit_information, nr_clusters):
    """
    Estimate orbit parameters from the given
    MC sample of 6D information for the Nr of stars
    and save it into orbit_information
    """

    for each_star in range(nr_clusters):
        
        # We are creating a dictionary for each star
        star_i = dict()
        
        ra     = MC_sample_6D['ra'][each_star]           *u.deg
        dec    = MC_sample_6D['dec'][each_star]          *u.deg
        dist   = MC_sample_6D['distance'][each_star]     *u.kpc
        pm_ra  = MC_sample_6D['pmra'][each_star]         *u.mas/u.year
        pm_dec = MC_sample_6D['pmdec'][each_star]        *u.mas/u.year
        v_los  = MC_sample_6D['vrad'][each_star]         *u.km/u.s

        # Create the Orbit instance
        o = Orbit(
            vxvv=[ra,dec,dist,pm_ra, pm_dec,v_los],
            ro=_REFR0*u.kpc,
            vo=_REFV0*u.km/u.s,
            zo=Z_gc_sun_kpc*u.kpc,
            solarmotion='schoenrich',
            radec=True
        )
        
        #Galactocentric coordinates:
        star_i['X_XYZ'] = o.x()#*u.kpc        
        star_i['Y_XYZ'] = o.y()#*u.kpc
        star_i['Z_XYZ'] = o.z()#*u.kpc
        star_i['U_LSR'] = o.U()#*u.km/u.s
        star_i['V_LSR'] = o.V()#*u.km/u.s
        star_i['W_LSR'] = o.W()#*u.km/u.s
        star_i['R_Rzphi'] = o.R()#*u.kpc
        star_i['phi_Rzphi'] = o.phi()#*u.rad
        star_i['z_Rzphi'] = o.z()#*u.kpc
        star_i['vR_Rzphi'] = o.vR()#*u.km/u.s
        star_i['vphi_Rzphi'] = o.vT()#*u.km/u.s        
        star_i['vz_Rzphi'] = o.vz()#*u.km/u.s
        star_i['vT_Rzphi'] = o.vT()#*u.km/u.s
        
        try:
            star_i['J_R'], star_i['L_Z'],star_i['J_Z'] = aAS(
                #R,vR,vT,z,vz[,phi]
                star_i['R_Rzphi'],
                star_i['vR_Rzphi'],
                star_i['vT_Rzphi'],
                star_i['z_Rzphi'],
                star_i['vz_Rzphi'],
                star_i['phi_Rzphi'],
                ro=_REFR0*u.kpc,vo=_REFV0*u.km/u.s
            )
        except:
            star_i['J_R'] = [np.nan]
            star_i['L_Z'] = [np.nan]
            star_i['J_Z'] = [np.nan]

        try:
            star_i['ecc'], star_i['zmax'], star_i['R_peri'], star_i['R_ap'] = aAS.EccZmaxRperiRap(
                #R,vR,vT,z,vz[,phi]
                star_i['R_Rzphi'],
                star_i['vR_Rzphi'],
                star_i['vT_Rzphi'],
                star_i['z_Rzphi'],
                star_i['vz_Rzphi'],
                star_i['phi_Rzphi'],
                ro=_REFR0*u.kpc,vo=_REFV0*u.km/u.s,zo=Z_gc_sun_kpc*u.kpc
            )            
            star_i['zmax']
            star_i['R_peri']
            star_i['R_peri']
            
        except:
            star_i['ecc'] = [np.nan]
            star_i['zmax'] = [np.nan]
            star_i['R_peri'] = [np.nan]
            star_i['R_ap'] = [np.nan]

        star_i['Energy'] = o.E(pot=pot,ro=_REFR0*u.kpc,vo=_REFV0*u.km/u.s,zo=Z_gc_sun_kpc*u.kpc)
        star_i['Rguid'] = o.rguiding(pot=pot,ro=_REFR0*u.kpc,vo=_REFV0*u.km/u.s,zo=Z_gc_sun_kpc*u.kpc)

        for each_label in orbit_labels:
            if len(MC_sample_6D['ra'][0]) == 1:
                try:
                    orbit_information[each_label][each_star] = star_i[each_label][0]
                except:
                    print('did not work for '+each_label)
            else:
                try:
                    orbit_information[each_label] = np.array([np.nanmean(star_i[each_label][each_star,:]) for each_star in range(nr_clusters)])
                    orbit_information[each_label+'_err'] = np.array([np.std(star_i[each_label][each_star,:]) for each_star in range(nr_clusters)])

                    percentiles = np.percentile(star_i[each_label], q=[5,50,95])
                    orbit_information[each_label+'_5'][each_star] = percentiles[0]
                    orbit_information[each_label+'_50'][each_star] = percentiles[1]
                    orbit_information[each_label+'_95'][each_star] = percentiles[2]
                except:
                    print('did not work for '+each_label)
    return orbit_information


In [ ]:
# We will first 'sample' only once with the best value
#MC_sample_6D = sample_6d_uncertainty(six_dimensions,e_six_dimensions,MC_size=1)
#orbit_information = estimate_orbit_parameters(MC_sample_6D, orbit_information, nr_clusters)

# And now we sample with a certain Monte Carlo sampling size
MC_sample_6D = sample_6d_uncertainty(six_dimensions,e_six_dimensions,MC_size=MC_size)
orbit_information = estimate_orbit_parameters(MC_sample_6D, orbit_information, nr_clusters)

os.system('say "your program has finished"')


did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work

did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work

did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work

did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work

did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work

did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work

did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work

did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work

did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work

did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work for X_XYZ
did not work for Y_XYZ
did not work for Z_XYZ
did not work for U_LSR
did not work for V_LSR
did not work for W_LSR
did not work for R_Rzphi
did not work for phi_Rzphi
did not work for z_Rzphi
did not work for vR_Rzphi
did not work for vphi_Rzphi
did not work for vz_Rzphi
did not work for J_R
did not work for L_Z
did not work for J_Z
did not work for ecc
did not work for zmax
did not work for R_peri
did not work for R_ap
did not work for Energy
did not work for Rguid
did not work

In [39]:
orbit_information_pd = pd.DataFrame(orbit_information,columns=orbit_information.keys())
orbit_information_pd['name'] = clusters["Cluster"]
clusters_pd = clusters.to_pandas()
clusters_pd = clusters_pd.drop(['col0'], axis=1)
clusters_pd['name'] = clusters["Cluster"]

final_table = pd.merge(clusters_pd,orbit_information_pd, on='name')
final_table = final_table.drop(['name'], axis=1)
final_table.to_csv(file_directory+'clusters_orbits.csv')
